In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
def start_driver(url):
    option = Options()
    option.add_argument("--disable-infobars") 

    driver = webdriver.Chrome(options = option)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html)
    return driver, soup

In [3]:
def get_parsed_data(base_url, category):
    
    url = base_url+category
    driver, soup = start_driver(url)

    region=[]
    data_sku=[]
    title=[]
    href=[]
    price=[]
    promo_price=[]
    brand=[]
    
    nav = soup.find('nav')
    num_pages = len(nav.find_all('a'))-1

    for j in range(2):
        n = 0
        m = 0
        for i in range(num_pages-1):
            for div in soup.find_all('div',class_="catalog-2-level-product-card product-card subcategory-or-type__products-item with-rating with-prices-drop"):
                n += 1
                data_sku.append(div['data-sku'])
                href.append(div.a['href'])
                title.append(div.a['title'])
                
                for div_price in div.find_all('div',class_="product-unit-prices__actual-wrapper"):
                    try:
                        promo_price.append(div_price.find('span',class_="product-price__sum-rubles").get_text(strip=True)
                                +div.find('span',class_="product-price__sum-penny").get_text(strip=True))
                    except:
                        promo_price.append(div_price.find('span',class_="product-price__sum-rubles").get_text(strip=True))
                        
                for div_old_price in div.find_all('div',class_="product-unit-prices__old-wrapper"):
                    try:
                        try:
                            price.append(div_old_price.find('span',class_="product-price__sum-rubles").get_text(strip=True)
                                    +div.find('span',class_="product-price__sum-penny").get_text(strip=True))
                        except:
                            price.append(div_old_price.find('span',class_="product-price__sum-rubles").get_text(strip=True))
                    except:
                        try:
                            price.append(div_price.find('span',class_="product-price__sum-rubles").get_text(strip=True)
                                    +div.find('span',class_="product-price__sum-penny").get_text(strip=True))
                        except:
                            price.append(div_price.find('span',class_="product-price__sum-rubles").get_text(strip=True))
            if i != 6:
                driver.get(url+f'?page={i+2}')
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html)  
        
        for link in href[m:n]:
            driver.get(base_url+link)
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html)
            try:
                brand.append(soup.find('a',class_="product-attributes__list-item-link reset-link active-blue-text").get_text(strip=True)) 
            except:
                brand.append(None)
        m += n
            
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        if len(region) == 0:
            for i in range(n):
                region.append('MSK')
        else:
            for i in range(n):
                region.append('SPB')
         
        if j == 0:
            time.sleep(2)      
            driver.find_element(By.XPATH, "//address[@class='header-address__receive-address']").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "//p[contains(text(),'Самовывоз')]").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "//div[@class='multiselect__select']").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "//input[@placeholder='Выберите Город']").send_keys("Санкт-Петербург")
            time.sleep(2)
            driver.find_element(By.XPATH, "//span[contains(text(),'Санкт-Петербург')]").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "//button[@data-qa='button-save-pickup-address']//span[contains(text(),'Сохранить')]").click()
            time.sleep(2)
    
    for i in range(len(href)):
        href[i] = base_url+href[i]
    
    df = pd.DataFrame({"Регион":region, "id товара из сайта/приложения":data_sku, "Наименование":title, "Ссылка на товар":href, "Регулярная цена":price, "Промо цена":promo_price, "Бренд":brand})
    return df

In [4]:
base_url = "https://online.metro-cc.ru"
category = "/category/ovoshchi-i-frukty/ovoshchi"

In [5]:
df = get_parsed_data(base_url, category)
df.to_excel('parsed_data.xlsx')

In [6]:
df

,Регион,id товара из сайта/приложения,Наименование,Ссылка на товар,Регулярная цена,Промо цена,Бренд
0,MSK,481700,Огурцы гладкие,https://online.metro-cc.ru/products/ogurcy-gla...,130,130,Без бренда
1,MSK,557206,Огурцы короткоплодные,https://online.metro-cc.ru/products/ogurcy-kor...,229.55,229.55,Без бренда
2,MSK,567204,Томаты весовые,https://online.metro-cc.ru/products/tomaty-567204,246,246,Без бренда
3,MSK,470306,Картофель весовой,https://online.metro-cc.ru/products/kartofel-4...,17.90,17.90,Без бренда
4,MSK,238152,"Томаты черри, 250г",https://online.metro-cc.ru/products/tomaty-che...,116.90,99.90,Без бренда
...,...,...,...,...,...,...,...
198,SPB,481692,Капуста цветная разноцветная,https://online.metro-cc.ru/products/kapusta-cv...,279.05,279.05,ARTFRUIT
199,SPB,296710,Томат Сеньор помидор,https://online.metro-cc.ru/products/tomaty-sen...,239.05,239.05,Без бренда
200,SPB,470385,Редька весовая,https://online.metro-cc.ru/products/redka-470385,69.55,69.55,Без бренда
201,SPB,118928,METRO Chef Огурцы среднеплодные,https://online.metro-cc.ru/products/ogurcy-sre...,119.05,119.05,Без бренда
